In [ ]:
from math import ceil
from scanpath_graph import ScanPathGraph
from iou_graph import IOUGraph
from euclidean_graph import EuclideanGraph

from reflacx_graph_dataset import generate_dataset

In [ ]:
full_meta_path = '../reflacx_lib/full_meta.json' # if file doesn't exist, it will be created
reflacx_dir = "../data/reflacx"
mimic_dir = "../data/mimic/reflacx_imgs"

from metadata import Metadata

metadata = Metadata(reflacx_dir, mimic_dir, full_meta_path, max_dicom_lib_ram_percent=30)

#### Generating Dataset

In [ ]:
print('scanpath dataset:')
name = 'reflacx_densnet225_scanpath'
generate_dataset(name,
                 metadata,
                 graph_class=ScanPathGraph,
                 mean_features_fpath='avg_DensNet_REFLACX_features.npy',
                 log_dir='./log/{}'.format(name))

In [ ]:
print('iou dataset:')
name = 'reflacx_densnet225_iou'
generate_dataset(name,
                 metadata,
                 graph_class=IOUGraph,
                 mean_features_fpath='avg_DensNet_REFLACX_features.npy',
                 log_dir='./log/{}'.format(name))

print('euclidean dataset:')
name = 'reflacx_densnet225_euclidean'
generate_dataset(name,
                 metadata,
                 graph_class=EuclideanGraph,
                 mean_features_fpath='avg_DensNet_REFLACX_features.npy',
                 log_dir='./log/{}'.format(name))

#### Testing Dataset

In [ ]:
import dgl

In [ ]:
scp_pth = 'reflacx_densnet225_scanpath'
iou_pth = 'reflacx_densnet225_iou'
euc_pth = 'reflacx_densnet225_euclidean'

In [ ]:
scp_ds = dgl.data.CSVDataset(scp_pth)
len(scp_ds)

In [ ]:
iou_ds = dgl.data.CSVDataset(iou_pth)
len(iou_ds)

In [ ]:
euc_ds = dgl.data.CSVDataset(euc_pth)
len(euc_ds)